In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import sys 
path='../src/'
if path not in sys.path: 
    sys.path.insert(1,path)
from utils_OB import *
from preposessing import *

%load_ext autoreload
%autoreload 2

In [ ]:
poles={}
poles_f_min={}
poles_xi_min={}
poles_mac_min={}
test_fac=self.dist_fac           #used to distribute the data more evenly in the interval 0- 1
for key in modes:                               #iterate all keys up to second to last one
    if key==list(modes.keys())[-1]:
        break
    for i,mode1 in enumerate(modes[key]):       # iterate on all modes in the modal order
        mac=np.array([])
        dF=np.array([])
        dXi=np.array([])
        for j, mode2 in enumerate(modes[key+1]):   # iterate over all modes in the order above
            mac=np.append(mac,MAC(mode1.v,mode2.v) )       #calculate MAC-number for all combinations
            dF=np.append(dF,rel_diff(mode1.f,mode2.f))        #relative difference in frequency
            dXi=np.append(dXi,rel_diff(mode1.xi,mode2.xi))     #relative difference in damping 
            if self.comb=='combined':
                checks=(dF*dXi*(1-mac))        #### THE COMBINATION scheme
                # not sure if this is the best way, but it makes sense to weigh dF 
                # more than the others, after looking at the initial plots. 

            elif self.comb=='combined_w':
                checks=(dF**0.3*dXi*(1-mac))        #### THE Weigthed COMBINATION scheme
                
            elif self.comb=='freq_based':
                checks =dF                         #### Choose based on lowest freq. 
                
            else:
                checks=(dF**0.3*dXi*(1-mac))       #### THE COMBINATION scheme
                print('invalid input for combination type')
                
        k=np.argmin(checks)              # index of the closest pole in the order above
        new_key=str(key)+'.'+str(i)       # format: modal_order.mode_number

        
        # poles[new_key]=[k,mac[k],dF[k],dXi[k]]    
        poles[new_key]=[k,(1-mac[k])**(test_fac),dF[k]**(test_fac),dXi[k]**test_fac]    

self.poles=poles

timer['After_pole_distances']=time.time()
#########################################################################
###### The first k-means round:    ######################################
#########################################################################

# a bit unnessesary list and dict operations her, could be streamlined som more
xdata=np.array([])
ydata=np.array([])
zdata=np.array([])
for key in poles.keys():
    xdata=np.append(xdata,poles[key][1]) #MAC
    ydata=np.append(ydata,poles[key][2]) #dF
    zdata=np.append(zdata,poles[key][3]) #dXi 
X=np.zeros((len(xdata),3))
n_groups=2            # want two groups
for i in range(len(xdata)):  #prepearing data want array of form [Nx3]
    X[i,0]=xdata[i]
    X[i,1]=ydata[i]
    X[i,2]=zdata[i]
    
cc,self.labels_1,inertia=sc.k_means(X, n_groups)  # Calling the k-means algorithm



### Deciding which index that are modes to keep. 
# ### assuming we want to keep the largest group for now. 
# ### migth make a better check here some time
# check=np.count_nonzero(self.labels_1)/len(self.labels_1)
# if check > 0.5: # it there are more 0 than 1 in the array
#     self.keep_index_1 = 1
# else:
#     self.keep_index_1 = 0

# New criteria based on product of frequencies: 
# This alows us to remove over 50% of the poles in this first k-means
cluster0=1
cluster1=1
for i,lab in enumerate(self.labels_1):
    if lab==0:
        cluster0*=xdata[i]
    else:
        cluster1*=xdata[i]
if cluster0 < cluster1:
    self.keep_index_1=0
else: 
    self.keep_index_1=1

#Creating a list of poles to keep and poles not to keep
index=0
modes_to_keep_1={}
modes_not_to_keep_1={}
for key in list(modes.keys())[:-1]:  
    modes_to_keep_1[key]=[]
    modes_not_to_keep_1[key]=[]
    for i,mode in enumerate(modes[key]):
        if self.labels_1[index]==self.keep_index_1:
            modes_to_keep_1[key].append(mode)
        else: 
            modes_not_to_keep_1[key].append(mode)

        index+=1
self.keep_kmeans_1=modes_to_keep_1
self.not_keep_kmeans_1=modes_not_to_keep_1

In [2]:
folder_path='D:/data_disk/resampl/'
names=os.listdir(folder_path)
names.sort()

def cov_ssi(names):
    for file_p in names:
        try:
            if file_p not in os.listdir('D:/data_disk/5hz_covssi/'):
                path=folder_path+file_p    
                orders=np.arange(50,200,2)
                depth=120
                ts=ts_via_cov_ssi_df(path,orders,depth)
                ts.save('D:/data_disk/5hz_covssi/')
        except Exception as a:
            print(file_p,'feil: ',a)
            continue

In [10]:
import threading
thread_1=threading.Thread(target=cov_ssi,args=[names[:1000]])
thread_2=threading.Thread(target=cov_ssi,args=[names[1000:]])

thread_1.start()
thread_2.start()

thread_1.run()
thread_2.run()
print('begge')


HB141M-2015-03-25_23-40-49.pkl feil:  Array must not contain infs or NaNs
HB141M-2015-03-26_00-12-30.pkl feil:  Array must not contain infs or NaNs
HB141M-2015-03-26_00-48-54.pkl feil:  Unable to allocate 314. MiB for an array with shape (14400, 2861) and data type float64
HB141M-2015-03-26_01-20-20.pkl feil:  Unable to allocate 314. MiB for an array with shape (14400, 2861) and data type float64
HB141M-2015-03-26_01-53-21.pkl feil:  Unable to allocate 314. MiB for an array with shape (14400, 2861) and data type float64
HB141M-2015-03-26_02-24-21.pkl feil:  Unable to allocate 314. MiB for an array with shape (14400, 2861) and data type float64
HB141M-2015-03-26_02-56-16.pkl feil:  Unable to allocate 314. MiB for an array with shape (14400, 2861) and data type float64
HB141M-2015-03-26_03-27-24.pkl feil:  Unable to allocate 314. MiB for an array with shape (14400, 2861) and data type float64
HB141M-2015-03-26_04-01-24.pkl feil:  Unable to allocate 314. MiB for an array with shape (14400

: 

In [3]:
files=['HB141M-2015-03-26_00-48-54.pkl','HB141M-2015-03-26_02-24-21.pkl']

cov_ssi(files)